## 0. Install Dependecies

In [ ]:
pip install torch

**Set device to GPU if is available otherwise set device as cpu**

In [ ]:
import torch
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Import libraries**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

##1. Dataset##

- The Datatset used in this project was generated using the Mujoco simulator with three different configurations:
- 2D (2 joints)
- 2D (3 joints)
- 3D (5 joints)

The format of the data is in CSV format, including information about Joint angles, fingertip position, and orientation.


1.1. Visualise data from the simulator

In [ ]:
!head -5 logfiler2.csv

j0;j1;cos(j0);cos(j1);sin(j0);sin(j1);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.998;  1.000;  0.055; -0.012;  0.210;  0.010;  1.000;  0.021
 0.076; -0.017;  0.997;  1.000;  0.076; -0.017;  0.210;  0.014;  1.000;  0.030
 0.148; -0.011;  0.989;  1.000;  0.147; -0.011;  0.208;  0.030;  0.998;  0.068
 0.214;  0.048;  0.977;  0.999;  0.212;  0.048;  0.204;  0.050;  0.991;  0.131


In [ ]:
!head -5 logfiler3.csv

j0;j1;j2;cos(j0);cos(j1);cos(j2);sin(j0);sin(j1);sin(j2);ft_x;ft_y;ft_qw;ft_qz
 0.055; -0.012;  0.072;  0.998;  1.000;  0.997;  0.055; -0.012;  0.072;  0.309;  0.022;  0.998;  0.057
 0.076; -0.017;  0.100;  0.997;  1.000;  0.995;  0.076; -0.017;  0.100;  0.308;  0.031;  0.997;  0.080
 0.135; -0.059;  0.194;  0.991;  0.998;  0.981;  0.135; -0.059;  0.193;  0.305;  0.050;  0.991;  0.135
 0.228; -0.110;  0.295;  0.974;  0.994;  0.957;  0.226; -0.109;  0.290;  0.297;  0.079;  0.979;  0.205


In [ ]:
!head -5 logfiler5.csv

j0;j1;j2;j3;j4;cos(j0);cos(j1);cos(j2);cos(j3);cos(j4);sin(j0);sin(j1);sin(j2);sin(j3);sin(j4);ft_x;ft_y;ft_z;ft_qw;ft_qx;ft_qy;ft_qz
 0.000;  0.000;  0.000;  0.000;  0.000;  1.000;  1.000;  1.000;  1.000;  1.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.000;  0.590;  1.000;  0.000;  0.000;  0.000
 0.022; -0.005;  0.028;  0.016; -0.032;  1.000;  1.000;  1.000;  1.000;  0.999;  0.022; -0.005;  0.028;  0.016; -0.032;  0.011;  0.004;  0.590;  1.000; -0.016;  0.019;  0.011
 0.103;  0.005;  0.107;  0.017; -0.100;  0.995;  1.000;  0.994;  1.000;  0.995;  0.102;  0.005;  0.106;  0.017; -0.099;  0.041;  0.016;  0.587;  0.995; -0.053;  0.061;  0.054
 0.209;  0.067;  0.216;  0.013; -0.174;  0.978;  0.998;  0.977;  1.000;  0.985;  0.208;  0.067;  0.215;  0.013; -0.173;  0.100;  0.042;  0.573;  0.979; -0.101;  0.138;  0.116


1.2. Preprocess the data


2R Robot

In [ ]:
# Load dataset
data = pd.read_csv('logfiler2.csv', delimiter=';')

# Preprocessing: Extract inputs (joint angles and their trigonometric functions) and outputs (fingertip positions and quaternions)
X = data[['j0', 'j1', 'cos(j0)', 'cos(j1)', 'sin(j0)', 'sin(j1)']].values
y = data[['ft_x', 'ft_y', 'ft_qw', 'ft_qz']].values

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

##2. Train Forward Kinematics Models##



### 2.1. Robot 2R

- Split the data into training and testing sets

In [ ]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

- Define the architecture of the model (Feedforward Neural Network) to learn forward kinematics.




In [ ]:
# Define the feedforward neural network model
class ForwardKinematicsNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ForwardKinematicsNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
# Model
model = ForwardKinematicsNN(input_dim=X_train.shape[1], hidden_dim=hidden_dim, output_dim=y_train.shape[1])

- Hyperparameter Search

Hyper-parameter tuning can be realized by using search or optimization methods.

[GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) is  a simple search routine to determine the best values of hyper-parameters among a set of possible predefined values. It uses cross-validation to measure performance of each set of hyper-parameters.

In the following section are showed 2 examples of GridSearch: one for classification and the other for regression.
Then in the cell below are shown the best parameters of each model.

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

#The first parameter is the model
grid_search_regression = GridSearchCV(model,
                           {
                              'C':np.arange(0.1,2,0.1),
                            'kernel': ('linear', 'rbf','poly','sigmoid'),
                            #If the kernel is poly (above line commented), you can also choose the best degree
                            #Otherwise keep commented
                            #'degree': np.arange(1,4,1),
                            },cv=5, scoring="r2",verbose=1,n_jobs=-1
                           )
print("\n\nGrid Search:\n\n")
print(grid_search_regression.fit(X_train,y_train))



Grid Search:




InvalidParameterError: The 'estimator' parameter of GridSearchCV must be an object implementing 'fit'. Got ForwardKinematicsNN(
  (fc1): Linear(in_features=6, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
  (relu): ReLU()
) instead.

In [ ]:
#Best parameters for the regressor
print("Best regression hyper-parameters: %r" %grid_search_regression.best_params_)
print("Best R2 score: %.2f" %grid_search_regression.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
# Hyperparameters
hidden_dim = 128  # Number of neurons in hidden layers
learning_rate = 0.01
batch_size = 64
epochs = 200
accuracy_threshold = 0.01  # Define accuracy threshold (e.g., 0.01 for close predictions)

- Loss Function

In [ ]:
criterion = nn.MSELoss()

- Choose a Solver (Optimizer)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

- Train the models on joint angle inputs to predict fingertip positions.

In [ ]:
# Lists to store losses and accuracies for plotting
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
# Accuracy function (percentage of predictions within a threshold)
def compute_accuracy(predictions, targets, threshold):
    with torch.no_grad():
        # Calculate absolute error
        error = torch.abs(predictions - targets)
        # Calculate how many predictions are within the threshold
        correct = (error < threshold).all(dim=1)  # Check if all dimensions are within the threshold
        return correct.float().mean().item()  # Mean accuracy

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Compute training accuracy
    train_accuracy = compute_accuracy(outputs, y_train, accuracy_threshold)

    # Validation step
    model.eval()
    val_outputs = model(X_val)
    val_loss = criterion(val_outputs, y_val)
    val_accuracy = compute_accuracy(val_outputs, y_val, accuracy_threshold)

    # Store losses and accuracies for plotting
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # Print every 10th epoch
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {loss.item():.4f}, Training Accuracy: {train_accuracy*100:.2f}%, "
              f"Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {val_accuracy*100:.2f}%")

Epoch [10/200], Training Loss: 0.0171, Training Accuracy: 0.00%, Validation Loss: 0.0136, Validation Accuracy: 0.00%
Epoch [20/200], Training Loss: 0.0050, Training Accuracy: 0.02%, Validation Loss: 0.0052, Validation Accuracy: 0.10%
Epoch [30/200], Training Loss: 0.0019, Training Accuracy: 0.10%, Validation Loss: 0.0016, Validation Accuracy: 0.19%
Epoch [40/200], Training Loss: 0.0007, Training Accuracy: 2.10%, Validation Loss: 0.0007, Validation Accuracy: 1.74%
Epoch [50/200], Training Loss: 0.0003, Training Accuracy: 6.34%, Validation Loss: 0.0003, Validation Accuracy: 4.02%
Epoch [60/200], Training Loss: 0.0002, Training Accuracy: 11.91%, Validation Loss: 0.0002, Validation Accuracy: 13.11%
Epoch [70/200], Training Loss: 0.0001, Training Accuracy: 23.25%, Validation Loss: 0.0001, Validation Accuracy: 23.85%
Epoch [80/200], Training Loss: 0.0001, Training Accuracy: 31.41%, Validation Loss: 0.0001, Validation Accuracy: 31.78%
Epoch [90/200], Training Loss: 0.0001, Training Accuracy: 

In [ ]:
# Testing the model
model.eval()
test_outputs = model(X_test)
test_loss = criterion(test_outputs, y_test)
test_accuracy = compute_accuracy(test_outputs, y_test, accuracy_threshold)
print(f"Test MSE: {test_loss.item():.4f}, Test Accuracy: {test_accuracy*100:.2f}%")

Test MSE: 0.0004, Test Accuracy: 0.70%


- plot the test and train loss and accuracy od the model

- Check real values and compare to true values of the dataset

In [ ]:
#import matplotlib
#from matplotlib import pyplot as plt

#matplotlib.rcParams['figure.figsize'] = [9,6]

#plt.plot(x, y, '.', label='Data')
#plt.plot(x, f(x), label='True fn')
#plt.legend()

##3. Compare Jacobians##



3.1. Compute the Jacobian matrix for the learned forward kinematics using automatic differentiation.



3.2. Compare the computed Jacobian with the analytical Jacobian for the 2-joint robot.